In [1]:
# get environment variables

In [8]:
run_to ='waom4'
run_from = 'waom10'

import os
import sys
from pandas import datetime
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from pyresample import kd_tree, geometry, utils

proj_dir = os.path.join(os.pardir,os.pardir)
data_dir = os.path.join(proj_dir,'data','preprocessing')

hr_grd_path = os.path.join(data_dir,'processed',run_to+'_grd.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(data_dir,'processed',run_from+'_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

lr_his_path =  os.path.join(proj_dir,'data','analysis','raw',run_from+'_M3','ocean_his_0006.nc')
lr_his = xr.open_dataset(lr_his_path).isel(ocean_time=0)


lr_ini_path = os.path.join(data_dir,'processed','waom10_ini.nc')
lr_ini = xr.open_dataset(lr_ini_path)

out_path = os.path.join(data_dir,'processed',run_to+'_ini.nc')

In [9]:
lr_def = {}
hr_def= {}

valid_input_index = {}
valid_output_index = {}
index_array = {}
distance_array = {}

for gt in ['rho','u','v']:

    lr_def[gt] = geometry.SwathDefinition(lons=lr_grd['lon_'+gt].values,lats=lr_grd['lat_'+gt].values)
    hr_def[gt] = geometry.SwathDefinition(lons=hr_grd['lon_'+gt].values,lats=hr_grd['lat_'+gt].values)

    valid_input_index[gt], valid_output_index[gt], index_array[gt], distance_array[gt] = \
    kd_tree.get_neighbour_info(lr_def[gt],hr_def[gt], 20000,neighbours=4,nprocs=16)


wf = lambda r: 1/r

/home/ubuntu/bigStick/anaconda3/envs/antarctic_melting/lib/python3.6/site-packages/pyresample/kd_tree.py:382: UserWarning: Possible more than 4 neighbours within 20000 m for some data points
  (neighbours, radius_of_influence))


In [10]:
def low_to_high(lr_da,lr_grd,hr_grd,gt,dim,fill_value=0.0):
    
    print('set up empty hr data array')
    if dim == 2:
    
        dummy = np.zeros(hr_grd['lon_'+gt].shape)
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        hr_da = xr.DataArray(dummy,coords=[y,x],dims=['eta_'+gt,'xi_'+gt])
        
    elif dim == 3:
        
        N = lr_da.s_rho.size
        dummy = np.tile(np.zeros(hr_grd['lon_'+gt].shape),(N,1,1))
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        z = lr_da['s_rho']
        hr_da = xr.DataArray(dummy,coords=[z,y,x],dims=['s_rho','eta_'+gt,'xi_'+gt])
    
    
    # Fill the mask of low resolution data with nearest neibghours and fill in known values on high res grid.
    if dim == 2:
        
        print('Fill in the mask of lr data')
        data = lr_da.values

        valid_mask = ~np.isnan(data)
        coords = np.array(np.nonzero(valid_mask)).T
        values = data[valid_mask]

        it = interpolate.NearestNDInterpolator(coords,values)

        filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
        
        print('Resample to high resolution grid')
        hr_da[:,:] = kd_tree.get_sample_from_neighbour_info('custom', hr_def[gt].shape, filled,\
                                             valid_input_index[gt],\
                                             valid_output_index[gt],index_array[gt],distance_array[gt],wf)
        
        # Fill with zeros where mask is present
        #print('fill hr mask areas with fill value: ',fill_value)
        #hr_da.values[hr_grd['mask_'+gt].values == 0] = fill_value
            
    if dim == 3:
        
        print('Fill in the mask of lr data and resample to high resolution grid')
        for k in np.arange(N):
            
            print('processing depth level: ',k)
            data = lr_da[k].values

            valid_mask = ~np.isnan(data)
            coords = np.array(np.nonzero(valid_mask)).T
            values = data[valid_mask]

            it = interpolate.NearestNDInterpolator(coords,values)

            filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
    
            # Fill in known values on high res grid
            hr_da[k] = kd_tree.get_sample_from_neighbour_info('custom', hr_def[gt].shape,filled,\
                                             valid_input_index[gt],\
                                             valid_output_index[gt],index_array[gt],distance_array[gt],wf)
            
            # Fill with zeros where mask is present
            #print('fill hr mask areas with fill value: ',fill_value)
            #hr_da[k].values[hr_grd['mask_'+gt].values == 0] = fill_value
            
    return hr_da

In [11]:
hr_ini = lr_ini.drop(['u','v','ubar','vbar','salt','temp','zeta','ocean_time'])

## Interpolate low resolution variables on high resolution grid

### Function call for: zeta, ubar, vbar and u, v, temp, salt 
Get 2D and 3D high resolution data and assign to prepared ini file.

In [16]:
for var,gt,dim in zip(['zeta','ubar','vbar','u','v','temp','salt'],
                       ['rho','u','v','u','v','rho','rho'],
                       [2,2,2,3,3,3,3]):
    print('processing: ',var)
    hr_ini[var] = low_to_high(lr_his[var],lr_grd,hr_grd,gt,dim)

processing:  zeta
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  ubar
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  vbar
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  u
set up empty hr data array
Fill in the mask of lr data and resample to high resolution grid
processing depth level:  0
processing depth level:  1
processing depth level:  2
processing depth level:  3
processing depth level:  4
processing depth level:  5
processing depth level:  6
processing depth level:  7
processing depth level:  8
processing depth level:  9
processing depth level:  10
processing depth level:  11
processing depth level:  12
processing depth level:  13
processing depth level:  14
processing depth level:  15
processing depth level:  16
processing depth level:  17
processing depth level:  18
processing depth level:  19
processing depth level:  

Assign time dimension to new data arrays with the ocean_time set to the one from the low resolution solution.

In [40]:
hr_ini = hr_ini.squeeze(drop=True)
hr_ini.coords['ocean_time'] =  lr_his.ocean_time
for var in ['zeta','ubar','vbar','u','v','temp','salt']:
    hr_ini[var] = hr_ini[var].expand_dims('ocean_time')


### Save new ini file
Compare high resolution and low resolution ini file for consistency and save as netcdf file making sure that ocean_time is saved as unlimited dimension.

In [45]:
hr_ini.to_netcdf(out_path,unlimited_dims='ocean_time')